In [1]:
import os
import numpy as np
import nibabel as nb


In [2]:
#img=nb.load('BP4T_rest1_1_denoised_z35.nii.gz')
#img=nb.load('BP4T_rest1_1_denoised.nii.gz')
#img=nb.load('BP4T_rest1_1_denoised_z20_49.nii.gz')

In [154]:
## from: https://gist.github.com/fabianp/9396204419c7b638d38f

"""
Partial Correlation in Python (clone of Matlab's partialcorr)
This uses the linear regression approach to compute the partial 
correlation (might be slow for a huge number of variables). The 
algorithm is detailed here:
    http://en.wikipedia.org/wiki/Partial_correlation#Using_linear_regression
Taking X and Y two variables of interest and Z the matrix with all the variable minus {X, Y},
the algorithm can be summarized as
    1) perform a normal linear least-squares regression with X as the target and Z as the predictor
    2) calculate the residuals in Step #1
    3) perform a normal linear least-squares regression with Y as the target and Z as the predictor
    4) calculate the residuals in Step #3
    5) calculate the correlation coefficient between the residuals from Steps #2 and #4; 
    The result is the partial correlation between X and Y while controlling for the effect of Z
Date: Nov 2014
Author: Fabian Pedregosa-Izquierdo, f@bianp.net
Testing: Valentina Borghesani, valentinaborghesani@gmail.com
"""

import numpy as np
from scipy import stats, linalg

def partial_corr(C):
    """
    Returns the sample linear partial correlation coefficients between pairs of variables in C, controlling 
    for the remaining variables in C.
    Parameters
    ----------
    C : array-like, shape (n, p)
        Array with the different variables. Each column of C is taken as a variable
    Returns
    -------
    P : array-like, shape (p, p)
        P[i, j] contains the partial correlation of C[:, i] and C[:, j] controlling
        for the remaining variables in C.
    """
    
    C = np.asarray(C)
    p = C.shape[1]
    P_corr = np.zeros((p, p), dtype=np.float)
    for i in range(p):
        P_corr[i, i] = 1
        for j in range(i+1, p):
            idx = np.ones(p, dtype=np.bool)
            idx[i] = False
            idx[j] = False
            beta_i = linalg.lstsq(C[:, idx], C[:, j])[0]
            beta_j = linalg.lstsq(C[:, idx], C[:, i])[0]

            res_j = C[:, j] - C[:, idx].dot( beta_i)
            res_i = C[:, i] - C[:, idx].dot(beta_j)
            
            corr = stats.pearsonr(res_i, res_j)[0]
            P_corr[i, j] = corr
            P_corr[j, i] = corr
        
    return P_corr

def calc_vox_pcorr_dirs(d,m,nbours):
    
    d_shape=np.shape(d)
    res=np.zeros(d_shape[0:3]+(np.shape(nbours)[0]*2,)) #x,y,z and the neighbourhoods as the 4th dim, mult*2 b/c +/-
    xdim=d_shape[0]
    ydim=d_shape[1]
    zdim=d_shape[2]
    for x in range(0+1,xdim-1):
        for y in range(0+1,ydim-1):
            for z in range(0+1,zdim-1):
                for idx,nbour in enumerate(nbours):
                    nidx = idx*2
                    loc = [x,y,z] + nbour
                    locm = [x,y,z] + nbour*-1
#                    corrmat=np.hstack((np.ndarray.flatten(d[tuple(loc)]).reshape(1,-1).T,
#                                   np.ndarray.flatten(d[x,y,z]).reshape(1,-1).T,
#                                   np.ndarray.flatten(d[tuple(locm)]).reshape(1,-1).T))
#                    res_corr=partial_corr(corrmat)
                    
#                    res[x,y,z,nidx]=res_corr[1,0] #pcorr btw the first two
#                    res[x,y,z,nidx+1]=res_corr[1,2] #pcorr btw the last two
                    
                    print(nidx)
    return res
                    

In [155]:
for idx,a in enumerate(range(1,10)):
    #print(idx)
    print(idx%2)
    

0
1
0
1
0
1
0
1
0


In [156]:
## setup neighbourhood matrices to use as loop for calculating the partial correlations

lval = 1 #value of voxels to lookup (or down, sideways etc), mult by n to increase range

# setup so that we can mult by -1 to give the other side of neighbourhood
neighbours_6=np.array([[lval, 0, 0],            [0,lval,0],             [0,0,lval]])
neighbours_12=np.array([[lval, 0, 0],           [0,lval,0],             [0,0,lval],
                       [lval,lval,0],           [lval,0,lval],          [0,lval,lval],
                       [lval,lval*-1,0],        [lval,0,lval*-1],       [0,lval,lval*-1]])
#currently wrong
neighbours_24=np.array([[lval, 0, 0],           [0,lval,0],             [0,0,lval],
                        [lval,lval,0],          [lval,0,lval],          [0,lval,lval],
                        [lval,lval*-1,0],       [lval,0,lval*-1],       [0,lval,lval*-1],
                        [lval,lval*-1,lval],    [lval,lval,lval*-1],    [lval,lval,lval],
                        [lval*-1,lval*-1,lval], [lval,lval*-1,lval*-1], [lval*-1,lval,lval*-1]])


In [158]:
calc_vox_pcorr_dirs(np.zeros((3,5,4)),[1],neighbours_6)
#np.zeros((3,5,4))[tuple([0,0,0])]


0
2
4
0
2
4
0
2
4
0
2
4
0
2
4
0
2
4


array([[[[ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.]],

        [[ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.]]],


       [[[ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.],
         [ 0.,  0.,  0.,  0.,  0.,  0.]],

  

In [9]:
d=img.get_data() #this takes a while on a full image
a=img.affine
mask=d[:,:,:,0]
mask[np.logical_not(mask==0)]=1

In [10]:
np.sum(mask)

7322.0

In [11]:
xdim=np.shape(d)[0]
ydim=np.shape(d)[1]
zdim=np.shape(d)[2] #not used here, since I only loaded a single slice to check this
print xdim,ydim,zdim
print ""

128 128 1



In [ ]:
nbours=6
res = np.zeros(np.shape(d)[0:3]+(nbours,)) #x,y,z and the neighbourhoods
print(np.shape(res))
# TODO: get rid of enumerate
print("x slice:")
for idxx,x in enumerate(range(0+1,xdim-1)):
    print x
    for idxy,y in enumerate(range(0+1,ydim-1)):
        for idxz,z in enumerate(range(0+1,zdim-1)):
        #    if x<0 or y<0 or z<0 or x>xdim-1 or y>ydim-1 or z>zdim-1:
        #        pass
        #    else:
            if not(mask[x,y,z]==0):
                #print x,y,z
                #for now, 2 directions of correlations
                corrmat=np.hstack((np.ndarray.flatten(d[x-1,y,z]).reshape(1,-1).T,
                                   np.ndarray.flatten(d[x,y,z]).reshape(1,-1).T,
                                   np.ndarray.flatten(d[x+1,y,z]).reshape(1,-1).T))
                res_corr=partial_corr(corrmat)
                res[x,y,z,0]=res_corr[1,0] #pcorr btw the first two
                res[x,y,z,1]=res_corr[1,2] #pcorr btw the last two

                corrmat=np.hstack((np.ndarray.flatten(d[x,y-1,z]).reshape(1,-1).T,
                                   np.ndarray.flatten(d[x,y,z]).reshape(1,-1).T,
                                   np.ndarray.flatten(d[x,y+1,z]).reshape(1,-1).T))
                res_corr=partial_corr(corrmat)
                res[x,y,z,2]=res_corr[1,0] #pcorr btw the first two
                res[x,y,z,3]=res_corr[1,2] #pcorr btw the last two    

                corrmat=np.hstack((np.ndarray.flatten(d[x,y,z-1]).reshape(1,-1).T,
                                   np.ndarray.flatten(d[x,y,z]).reshape(1,-1).T,
                                   np.ndarray.flatten(d[x,y,z+1]).reshape(1,-1).T))
                res_corr=partial_corr(corrmat)
                res[x,y,z,4]=res_corr[1,0] #pcorr btw the first two
                res[x,y,z,5]=res_corr[1,2] #pcorr btw the last two 
print ""

In [ ]:
res[np.isnan(res)]=0
out_img=nb.Nifti1Image(res,a,header=img.get_header())
nb.save(out_img,'6neighbours.nii.gz')

Try for some parallel action?

def calc_vox_vectors(d,mask,x,y,z):
    #calc the partial corr between each pair in the neighbourhood, controlling for the diagonal other
    res=np.zeros((1,6))
    if not(mask[x,y,z]==0):
        corrmat=np.hstack((np.ndarray.flatten(d[x-1,y,z]).reshape(1,-1).T,
                           np.ndarray.flatten(d[x,y,z]).reshape(1,-1).T,
                           np.ndarray.flatten(d[x+1,y,z]).reshape(1,-1).T))
        res_corr=partial_corr(corrmat)
        res[0]=res_corr[0,1] #pcorr btw the first two
        res[1]=res_corr[1,2] #pcorr btw the last two

        corrmat=np.hstack((np.ndarray.flatten(d[x,y-1,z]).reshape(1,-1).T,
                           np.ndarray.flatten(d[x,y,z]).reshape(1,-1).T,
                           np.ndarray.flatten(d[x,y+1,z]).reshape(1,-1).T))
        res_corr=partial_corr(corrmat)
        res[2]=res_corr[0,1] #pcorr btw the first two
        res[3]=res_corr[1,2] #pcorr btw the last two    

        corrmat=np.hstack((np.ndarray.flatten(d[x,y,z-1]).reshape(1,-1).T,
                           np.ndarray.flatten(d[x,y,z]).reshape(1,-1).T,
                           np.ndarray.flatten(d[x,y,z+1]).reshape(1,-1).T))
        res_corr=partial_corr(corrmat)
        res[4]=res_corr[0,1] #pcorr btw the first two
        res[5]=res_corr[1,2] #pcorr btw the last two
    return res

#!/usr/bin/env python2
import itertools
from multiprocessing import Pool, freeze_support
img=nb.load('BP4T_rest1_1_denoised_z20_49.nii.gz')
d=img.get_data()



pool = Pool()
xargs=range(0+1,xdim-1)
yargs=range(0+1,ydim-1)
zargs=range(0+1,zdim-1)
a= pool.map(calc_vox_vectors, itertools.izip(d,mask,xargs,yargs,zargs))


In [28]:
a


[]